In [ ]:
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np
import xarray as xr
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from IPython.display import Image, display
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
pio.renderers.default="notebook"

In [ ]:
df = pd.read_csv('./data/df_final_merged.csv')

In [ ]:
df['time'] = pd.to_datetime(df['time'], format='mixed', errors='coerce')
df = df.sort_values("time")

In [ ]:
df.info()

In [ ]:
df.timedelta_ms

### Sudden Price Flag

In [ ]:
df['spot_sudden_flag'] = (
    (df['window_5_pct_change'] < -0.00587) |
    (df['window_5_pct_change'] > -0.00308)
).astype(int)

df['perp_sudden_flag'] = (
    (df['window_3_pct_change_perp'] < -0.001) |
    (df['window_3_pct_change_perp'] > 0.247)
).astype(int)

### Predicting Perpetual Based on Spot Market

In [ ]:
features = [
    'spot_sudden_flag',
    'timedelta_ms',
    'mid_price_spot',
    'price_dev',
    'rolling_signed_volume_3ms',
    'rolling_signed_volume_5ms',
    'trade_direction'
]
target = 'window_3_pct_change_perp'

# Drop rows with missing values
df_model = df[features + [target]].dropna().reset_index(drop=True)

# Scale features
scaler = StandardScaler()
X_data = scaler.fit_transform(df_model[features].values)
y_data = df_model[target].values

In [ ]:
with pm.Model() as model_5ms_t:
    # priors
    a = pm.Normal('a', mu=0, sigma=1)
    beta = pm.Normal('beta', mu=0, sigma=1, shape=X_data.shape[1])
    sigma = pm.HalfNormal('sigma', 1)

    # v_ = pm.Exponential("v_", 1/29) 
    # v = pm.Deterministic("v", v_+1) # weakly informative prior

    # linear model
    u = pm.Deterministic('mu', a+pm.math.dot(X_data, beta))

    # likelihood
    # y_pred = pm.StudentT('y', mu=u, sigma=sigma, nu=v, observed=y_data)
    y_pred = pm.Normal('y', mu=u, sigma=sigma, observed=y_data)

    # inference
    idata_5ms_t = pm.sample(idata_kwargs={'log_likelihood':True})
    idata_5ms_t.extend(pm.sample_posterior_predictive(idata_5ms_t))

In [ ]:
az.summary(idata_5ms_t, var_names=["a", "beta", "sigma"], hdi_prob=0.94)

In [ ]:
mu_mean = idata_5ms_t.posterior["mu"].mean(dim=("mu_dim_0"))  # shape: [chain, draw]
idata_5ms_t.posterior['posterior_mean'] = mu_mean

In [ ]:
az.plot_forest(idata_5ms_t, var_names=['a', 'beta'], combined=True, figsize=(10,4))
plt.plot()

In [ ]:
spot_population_mean_5pct_change = np.mean(df['window_3_pct_change_perp']) * 100
print(f"Mean percentage change of spot population: {spot_population_mean_5pct_change:.8f}%")

In [ ]:
az.plot_posterior(idata_5ms_t, var_names=['posterior_mean'], figsize=(10,6), ref_val=0.000956,hdi_prob=0.94)
plt.plot()

In [ ]:
y_pred_samples = idata_5ms_t.posterior_predictive["y"].stack(sample=("chain", "draw")).values
y_pred_mean = y_pred_samples.mean(axis=1)

rmse = np.sqrt(mean_squared_error(y_data, y_pred_mean))
print(f"RMSE: {rmse:.6f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_data[:300], label="Actual", alpha=0.7, color="C0")
plt.plot(y_pred_mean[:300], label="Predicted (mean)", alpha=0.7, color="C1")
plt.fill_between(
    range(300),
    np.percentile(y_pred_samples[:300], 3, axis=1),
    np.percentile(y_pred_samples[:300], 97, axis=1),
    color="C1",
    alpha=0.3,
    label="94% CI"
)
plt.title(f"Posterior Predictive vs Actual (First 300 samples) | RMSE: {rmse:.6f}")
plt.xlabel("Sample Index")
plt.ylabel("5ms % Change in Perpetual")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Predicting Spot Based on Perpetual Market

In [ ]:
features = [
    'perp_sudden_flag',
    'timedelta_ms_perp',
    'mid_price_perp',
    'trade_direction_perp'
]
target = 'window_5_pct_change'

# Drop rows with missing values
df_model = df[features + [target]].dropna().reset_index(drop=True)

# Scale features
scaler = StandardScaler()
X_data = scaler.fit_transform(df_model[features].values)
y_data = df_model[target].values

In [ ]:
with pm.Model() as model_5ms_t:
    # priors
    a = pm.Normal('a', mu=0, sigma=1)
    beta = pm.Normal('beta', mu=0, sigma=1, shape=X_data.shape[1])
    sigma = pm.HalfNormal('sigma', 1)

    # linear model
    u = pm.Deterministic('mu', a+pm.math.dot(X_data, beta))

    # likelihood
    y_pred = pm.Normal('y', mu=u, sigma=sigma, observed=y_data)

    # inference
    idata_5ms_t = pm.sample(idata_kwargs={'log_likelihood':True})
    idata_5ms_t.extend(pm.sample_posterior_predictive(idata_5ms_t))

In [ ]:
y_pred_samples = idata_5ms_t.posterior_predictive["y"].stack(sample=("chain", "draw")).values
y_pred_mean = y_pred_samples.mean(axis=1)

rmse = np.sqrt(mean_squared_error(y_data, y_pred_mean))
print(f"RMSE: {rmse:.6f}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_data[:300], label="Actual", alpha=0.7, color="C0")
plt.plot(y_pred_mean[:300], label="Predicted (mean)", alpha=0.7, color="C1")
plt.fill_between(
    range(300),
    np.percentile(y_pred_samples[:300], 3, axis=1),
    np.percentile(y_pred_samples[:300], 97, axis=1),
    color="C1",
    alpha=0.3,
    label="94% CI"
)
plt.title(f"Posterior Predictive vs Actual (First 300 samples) | RMSE: {rmse:.6f}")
plt.xlabel("Sample Index")
plt.ylabel("5ms % Change in Perpetual")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()